### prophet

In [1]:
import pandas as pd
from prophet import Prophet

/home/mehedi/miniconda3/envs/fa1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [3]:
train_data = pd.read_csv('snp_price_phophet_train.csv')
test_data = pd.read_csv('snp_price_phophet_test.csv')

In [4]:
train_data

,ds,stock,y
0,2018-01-03 00:00:00,A,1.025444
1,2018-01-04 00:00:00,A,1.017751
2,2018-01-05 00:00:00,A,1.034024
3,2018-01-08 00:00:00,A,1.036243
4,2018-01-09 00:00:00,A,1.061686
...,...,...,...
547259,2022-04-22 00:00:00,ZTS,2.604193
547260,2022-04-25 00:00:00,ZTS,2.580577
547261,2022-04-26 00:00:00,ZTS,2.532916
547262,2022-04-27 00:00:00,ZTS,2.554385


In [5]:
test_data

,ds,stock,y
0,2022-04-29 00:00:00,A,1.823123
1,2022-05-02 00:00:00,A,1.827709
2,2022-05-03 00:00:00,A,1.870967
3,2022-05-04 00:00:00,A,1.930581
4,2022-05-05 00:00:00,A,1.867146
...,...,...,...
136811,2023-05-23 00:00:00,ZTS,2.492338
136812,2023-05-24 00:00:00,ZTS,2.441386
136813,2023-05-25 00:00:00,ZTS,2.385093
136814,2023-05-26 00:00:00,ZTS,2.377876


In [6]:
prophet_models = {}
for stock in train_data['stock'].unique():
    # Filter data for the specific stock
    df_stock = train_data[train_data['stock'] == stock]

    # Create a new Prophet model for the stock
    model = Prophet()

    # Fit the model to the stock's data
    model.fit(df_stock)

    # Store the fitted model in the dictionary
    prophet_models[stock] = model

11:55:57 - cmdstanpy - INFO - Chain [1] start processing
11:55:58 - cmdstanpy - INFO - Chain [1] done processing
11:55:58 - cmdstanpy - INFO - Chain [1] start processing
11:55:58 - cmdstanpy - INFO - Chain [1] done processing
11:55:58 - cmdstanpy - INFO - Chain [1] start processing
11:55:59 - cmdstanpy - INFO - Chain [1] done processing
11:55:59 - cmdstanpy - INFO - Chain [1] start processing
11:56:00 - cmdstanpy - INFO - Chain [1] done processing
11:56:00 - cmdstanpy - INFO - Chain [1] start processing
11:56:00 - cmdstanpy - INFO - Chain [1] done processing
11:56:00 - cmdstanpy - INFO - Chain [1] start processing
11:56:01 - cmdstanpy - INFO - Chain [1] done processing
11:56:01 - cmdstanpy - INFO - Chain [1] start processing
11:56:02 - cmdstanpy - INFO - Chain [1] done processing
11:56:02 - cmdstanpy - INFO - Chain [1] start processing
11:56:02 - cmdstanpy - INFO - Chain [1] done processing
11:56:03 - cmdstanpy - INFO - Chain [1] start processing
11:56:03 - cmdstanpy - INFO - Chain [1]

In [7]:
stock_list =[]
for x in test_data['stock'].unique():
    stock_list.append(x)

prediction dataframe 

In [8]:
prophet_pred = pd.DataFrame(columns=['ds'] + stock_list)  # Initialize the forecasted values dataframe

for stock in stock_list:
    df_stock_test = test_data[test_data['stock'] == stock]
    model = prophet_models[stock]
    future = model.make_future_dataframe(periods=len(df_stock_test))
    forecast = model.predict(future)
    predicted_values = forecast[['ds', 'yhat']].tail(len(df_stock_test))
    
    # Rename the 'yhat' column to the stock symbol
    predicted_values = predicted_values.rename(columns={'yhat': stock})
    
    prophet_pred = pd.merge(prophet_pred, predicted_values, on='ds', how='outer')  # Merge the predicted values with the main dataframe

prophet_pred['ds'] = pd.to_datetime(prophet_pred['ds'])  # Convert 'ds' column to datetime if not already

# Sort the dataframe based on the 'ds' column
prophet_pred = prophet_pred.sort_values('ds').reset_index(drop=True)

# Print the forecasted values dataframe
print(prophet_pred)


     A_x AAL_x AAP_x AAPL_x ABBV_x ABC_x ABT_x ACGL_x ACN_x ADBE_x  ...  \
0    NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN   NaN    NaN  ...   
1    NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN   NaN    NaN  ...   
2    NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN   NaN    NaN  ...   
3    NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN   NaN    NaN  ...   
4    NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN   NaN    NaN  ...   
..   ...   ...   ...    ...    ...   ...   ...    ...   ...    ...  ...   
267  NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN   NaN    NaN  ...   
268  NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN   NaN    NaN  ...   
269  NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN   NaN    NaN  ...   
270  NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN   NaN    NaN  ...   
271  NaN   NaN   NaN    NaN    NaN   NaN   NaN    NaN   NaN    NaN  ...   

       WYNN_y     XEL_y     XOM_y    XRAY_y     XYL_y     YUM_y     ZBH_y  \
0    0.519746  1.70834

In [9]:
prophet_pred = prophet_pred.loc[:, ~prophet_pred.columns.str.endswith('_x')]

# Rename remaining columns to remove '_y' suffix and keep only stock names
prophet_pred.columns = prophet_pred.columns.str.replace('_y', '')

# Print the modified dataframe
print(prophet_pred)

            ds         A       AAL       AAP      AAPL      ABBV       ABC  \
0   2022-04-29  1.977556  0.301744  2.176896  4.137167  2.006712  1.806538   
1   2022-04-30  1.951938  0.261104  2.165170  4.212221  1.926960  1.736530   
2   2022-05-01  1.951174  0.259805  2.166557  4.214890  1.932725  1.737538   
3   2022-05-02  1.971126  0.299528  2.180363  4.144673  2.022752  1.807990   
4   2022-05-03  1.969958  0.299132  2.186592  4.153666  2.027265  1.809611   
..         ...       ...       ...       ...       ...       ...       ...   
267 2023-01-21  1.928109  0.176870  2.517608  5.196478  2.429723  2.089061   
268 2023-01-22  1.924286  0.176866  2.513704  5.199494  2.432911  2.092544   
269 2023-01-23  1.941241  0.217953  2.522129  5.130015  2.520183  2.165527   
270 2023-01-24  1.937172  0.219007  2.522925  5.140128  2.521804  2.169727   
271 2023-01-25  1.933241  0.218276  2.515912  5.145230  2.525497  2.172903   

          ABT      ACGL       ACN  ...      WYNN       XEL     

In [10]:
prophet_pred.set_index('ds', inplace=True)  # Set 'ds' column as the index
prophet_pred.index.name = 'Date'  # Rename the index to 'Date'

# Print the modified dataframe
print(prophet_pred)

                   A       AAL       AAP      AAPL      ABBV       ABC  \
Date                                                                     
2022-04-29  1.977556  0.301744  2.176896  4.137167  2.006712  1.806538   
2022-04-30  1.951938  0.261104  2.165170  4.212221  1.926960  1.736530   
2022-05-01  1.951174  0.259805  2.166557  4.214890  1.932725  1.737538   
2022-05-02  1.971126  0.299528  2.180363  4.144673  2.022752  1.807990   
2022-05-03  1.969958  0.299132  2.186592  4.153666  2.027265  1.809611   
...              ...       ...       ...       ...       ...       ...   
2023-01-21  1.928109  0.176870  2.517608  5.196478  2.429723  2.089061   
2023-01-22  1.924286  0.176866  2.513704  5.199494  2.432911  2.092544   
2023-01-23  1.941241  0.217953  2.522129  5.130015  2.520183  2.165527   
2023-01-24  1.937172  0.219007  2.522925  5.140128  2.521804  2.169727   
2023-01-25  1.933241  0.218276  2.515912  5.145230  2.525497  2.172903   

                 ABT      ACGL       

now I keep it save for furthur use

In [ ]:
prophet_pred.to_csv('price_prophet_pred.csv',index=True)

### AutoTS

In [2]:
train_data = pd.read_csv('snp_price_AT_train.csv',index_col=0)
test_data = pd.read_csv('snp_price_AT_test.csv',index_col=0)

In [3]:
train_data

,ds,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
0,2018-01-03 00:00:00,1.025444,0.987734,1.009049,0.999826,1.015649,1.003722,1.002211,1.000906,1.004615,...,0.989166,0.993307,1.019640,0.996574,1.012193,0.999142,1.006932,1.019863,0.998817,1.004598
1,2018-01-04 00:00:00,1.017751,0.993961,1.046281,1.004470,1.009856,1.001489,1.000510,1.004643,1.016511,...,0.994522,0.985568,1.021051,0.996425,1.018951,1.009314,1.005481,1.040015,1.002959,1.010590
2,2018-01-05 00:00:00,1.034024,0.993584,1.057404,1.015906,1.027436,1.013611,1.003402,1.000680,1.024896,...,1.001156,0.978666,1.020228,1.010426,1.017041,1.015196,1.015476,1.056214,1.003353,1.022154
3,2018-01-08 00:00:00,1.036243,0.983771,1.049958,1.012133,1.010974,1.030413,1.000510,1.000793,1.033086,...,0.987827,0.985986,1.024815,1.017277,1.020714,1.016912,1.017411,1.066725,0.998422,1.034415
4,2018-01-09 00:00:00,1.061686,0.982827,1.041474,1.012017,1.018596,1.037005,1.002211,0.987882,1.036531,...,0.994522,0.974482,1.020463,0.974828,1.021007,1.014216,1.001048,1.099412,1.021894,1.046538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083,2022-04-22 00:00:00,1.854459,0.390725,2.129008,3.933075,1.948311,1.817675,2.189354,1.607361,2.154221,...,0.473068,1.753641,1.266299,0.631369,1.308077,1.615605,1.077834,3.644586,1.343576,2.604193
1084,2022-04-25 00:00:00,1.856446,0.386853,2.182431,3.959573,1.964779,1.811616,2.196125,1.587656,2.195947,...,0.466670,1.739480,1.223608,0.640311,1.298498,1.600689,1.095926,3.750265,1.351318,2.580577
1085,2022-04-26 00:00:00,1.802793,0.363425,2.178239,3.811770,1.963270,1.797669,2.140494,1.580181,2.100850,...,0.440884,1.731927,1.224055,0.629364,1.248443,1.540236,1.064543,3.611031,1.234967,2.532916
1086,2022-04-27 00:00:00,1.798819,0.367297,2.192277,3.806179,1.981372,1.800069,2.135919,1.628426,2.100850,...,0.439139,1.722250,1.259010,0.621193,1.268527,1.543008,1.053653,3.578343,1.238948,2.554385


In [4]:
test_data

,ds,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
0,2022-04-29 00:00:00,1.823123,0.363425,1.946123,3.832433,1.846364,1.729534,2.076995,1.551642,2.081859,...,0.455490,1.729095,1.268084,0.616568,1.243654,1.544460,1.035389,3.564362,1.250008,2.536923
1,2022-05-02 00:00:00,1.827709,0.359359,1.954507,3.839969,1.858809,1.703013,2.064734,1.558777,2.111247,...,0.463439,1.715170,1.285339,0.611325,1.264201,1.527961,1.034532,3.659917,1.261068,2.499996
2,2022-05-03 00:00:00,1.870967,0.370395,1.948951,3.876919,1.880556,1.723475,2.057414,1.577463,2.112772,...,0.463310,1.702425,1.311816,0.627206,1.272544,1.518853,1.034789,3.549706,1.271244,2.506723
3,2022-05-04 00:00:00,1.930581,0.379882,2.024113,4.035906,1.905571,1.792410,2.117254,1.607361,2.182361,...,0.465055,1.732400,1.364027,0.646787,1.395055,1.563995,1.083150,3.547102,1.315705,2.531341
4,2022-05-05 00:00:00,1.867146,0.358972,1.962696,3.811041,1.912988,1.774119,2.068028,1.578143,2.070353,...,0.442629,1.718002,1.343351,0.629210,1.332950,1.526113,1.050738,3.353100,1.269916,2.453909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,2023-05-23 00:00:00,1.979404,0.272423,1.185566,4.201650,1.861044,1.987738,1.971407,2.485957,2.029067,...,0.659914,1.592851,1.654185,0.595816,1.629442,1.768340,1.126755,2.615177,0.681566,2.492338
268,2023-05-24 00:00:00,1.861692,0.267196,1.169830,4.208507,1.843154,1.985416,1.939712,2.438392,2.020052,...,0.654797,1.581680,1.672686,0.581410,1.581250,1.760537,1.112057,2.520394,0.671826,2.441386
269,2023-05-25 00:00:00,1.838612,0.278426,1.133181,4.236672,1.810117,1.975087,1.904289,2.459117,2.054844,...,0.654344,1.561281,1.642681,0.577965,1.547297,1.752331,1.104967,2.504002,0.656752,2.385093
270,2023-05-26 00:00:00,1.852922,0.277845,1.138562,4.296429,1.796275,1.954659,1.917899,2.412231,2.138232,...,0.655056,1.562009,1.631953,0.578748,1.544000,1.755102,1.103670,2.552502,0.664637,2.377876


In [5]:
train_data.index=pd.to_datetime(train_data.index)
test_data.index=pd.to_datetime(test_data.index)

now I will try to run the model in fast mode

In [6]:
import numpy as np
import pandas as pd
from autots import AutoTS



# Create a new AutoTS model
model = AutoTS(
    forecast_length=len(test_data),
    frequency='infer',
    prediction_interval=0.9,
    ensemble=None,
    model_list="fast",
    transformer_list="fast",
    drop_most_recent=1,
    max_generations=4,
    num_validations=2,
    validation_method="backwards"
)

# Fit the AutoTS model
model = model.fit(train_data)




Inferred frequency is: N
Categorical features converted to numeric
Model Number: 1 with model AverageValueNaive in generation 0 of 4
Model Number: 2 with model AverageValueNaive in generation 0 of 4
Model Number: 3 with model AverageValueNaive in generation 0 of 4
Model Number: 4 with model DatepartRegression in generation 0 of 4
Model Number: 5 with model DatepartRegression in generation 0 of 4


/home/mehedi/miniconda3/envs/fa1/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/mehedi/miniconda3/envs/fa1/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/mehedi/miniconda3/envs/fa1/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/mehedi/miniconda3/envs/fa1/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  war

Model Number: 6 with model DatepartRegression in generation 0 of 4
Model Number: 7 with model DatepartRegression in generation 0 of 4


2023-07-05 17:37:31.101477: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-05 17:37:31.103192: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-05 17:37:31.135349: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-05 17:37:31.135605: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-05 17:37:31.634930: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

Epoch 1/50


...

Template Eval Error: Exception('Transformer RegressionFilter failed on fit') in model 67 in generation 0: UnivariateMotif
Model Number: 68 with model WindowRegression in generation 0 of 4
Model Number: 69 with model RRVAR in generation 0 of 4
Model Number: 70 with model KalmanStateSpace in generation 0 of 4
Model Number: 71 with model ETS in generation 0 of 4
Model Number: 72 with model MAR in generation 0 of 4
Template Eval Error: ValueError('Shape of passed values is (70, 504), indices imply (272, 504)') in model 72 in generation 0: MAR
Model Number: 73 with model RRVAR in generation 0 of 4
Template Eval Error: Exception('Transformer HolidayTransformer failed on fit') in model 73 in generation 0: RRVAR
Model Number: 74 with model RRVAR in generation 0 of 4
Model Number: 75 with model SectionalMotif in generation 0 of 4
Template Eval Error: Exception('Transformer AlignLastValue failed on inverse') in model 75 in generation 0: SectionalMotif
Model Number: 76 with model KalmanStateSpace

/home/mehedi/miniconda3/envs/fa1/lib/python3.9/site-packages/autots/tools/window_functions.py:468: RuntimeWarning: divide by zero encountered in divide
  slope = (sxy - sx * sy) / (sx2 - sx**2)
/home/mehedi/miniconda3/envs/fa1/lib/python3.9/site-packages/autots/tools/window_functions.py:468: RuntimeWarning: invalid value encountered in divide
  slope = (sxy - sx * sy) / (sx2 - sx**2)
/home/mehedi/miniconda3/envs/fa1/lib/python3.9/site-packages/numpy/core/_methods.py:180: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/home/mehedi/miniconda3/envs/fa1/lib/python3.9/site-packages/autots/tools/transform.py:3127: RuntimeWarning: invalid value encountered in add
  return df - (self.slope * self.dates_2d + self.intercept)


Template Eval Error: ValueError('more than 1 year of data is required for holiday detection.') in model 77 in generation 0: Cassandra
Model Number: 78 with model ETS in generation 0 of 4
Model Number: 79 with model ETS in generation 0 of 4
Template Eval Error: Exception('Transformer AlignLastValue failed on inverse') in model 79 in generation 0: ETS
Model Number: 80 with model ConstantNaive in generation 0 of 4
Model Number: 81 with model GLM in generation 0 of 4
Model Number: 82 with model ETS in generation 0 of 4
Model Number: 83 with model LastValueNaive in generation 0 of 4
Model Number: 84 with model AverageValueNaive in generation 0 of 4
Model Number: 85 with model WindowRegression in generation 0 of 4
Model Number: 86 with model MultivariateMotif in generation 0 of 4


: 

: 